# 🌀 Amazing Mazes — Benchmarks

Ce notebook compare **générateurs** (Backtracking, Kruskal) et **solveurs** (Backtracking, A*)
en mesurant temps, mémoire et caractéristiques du labyrinthe.

Il produit aussi un fichier CSV avec toutes les mesures.

In [ ]:
import importlib, time, tracemalloc, csv, os
from collections import Counter
from statistics import mean
import pandas as pd
import matplotlib.pyplot as plt

# Import des modules (assurez-vous que maze_generator.py et maze_solver.py sont dans le dossier)
gen_mod = importlib.import_module("maze_generator")
sol_mod = importlib.import_module("maze_solver")

WALL, EMPTY, PATH, SEEN = '#', '.', 'o', '*'

In [ ]:
def grid_to_str(grid):
    return "\n".join("".join(row) for row in grid)

def copy_grid(grid):
    return [row.copy() for row in grid]

def count_chars(grid):
    return dict(Counter(ch for row in grid for ch in row))

In [ ]:
def measure_generation(gen_fn, n, seed, label):
    start_cpu = time.process_time()
    start_wall = time.perf_counter()
    tracemalloc.start()
    grid = gen_fn(n, seed=seed)
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_wall = time.perf_counter()
    end_cpu = time.process_time()

    ascii_txt = grid_to_str(grid)
    H, W = len(grid), len(grid[0])
    return {
        "phase": "generation",
        "algo": label,
        "n": n,
        "seed": seed,
        "wall_time_s": end_wall - start_wall,
        "cpu_time_s": end_cpu - start_cpu,
        "peak_mem_bytes": peak,
        "ascii_bytes": len(ascii_txt.encode("utf-8")),
        "H": H,
        "W": W,
        "grid_cells": H*W
    }, grid

In [ ]:
def measure_solving(solve_fn, grid_in, n, seed, label, gen_label):
    g = copy_grid(grid_in)
    start_cpu = time.process_time()
    start_wall = time.perf_counter()
    tracemalloc.start()
    ok = solve_fn(g)
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_wall = time.perf_counter()
    end_cpu = time.process_time()

    counts = count_chars(g)
    return {
        "phase": "solving",
        "algo": label,
        "gen_algo": gen_label,
        "n": n,
        "seed": seed,
        "ok": bool(ok),
        "wall_time_s": end_wall - start_wall,
        "cpu_time_s": end_cpu - start_cpu,
        "peak_mem_bytes": peak,
        "path_len_cells": counts.get(PATH, 0),
        "explored_cells": counts.get(SEEN, 0),
        "remaining_empty_cells": counts.get(EMPTY, 0),
        "wall_cells": counts.get(WALL, 0)
    }, g

In [ ]:
# Paramètres d'expérience
sizes = [15, 25, 51]
reps = 2
seed_base = 123

generators = {
    "backtracking": gen_mod.carve_passages_recursive_backtracking,
    "kruskal": gen_mod.carve_maze_kruskal,
}
solvers = {
    "backtracking": sol_mod.solve_backtracking,
    "astar": sol_mod.solve_astar,
}

In [ ]:
rows = []

for n in sizes:
    for rep in range(reps):
        seed = seed_base + rep*1000 + n*17
        for gen_name, gen_fn in generators.items():
            gm, grid = measure_generation(gen_fn, n, seed, label=f"gen_{gen_name}")
            rows.append(gm)
            for sol_name, sol_fn in solvers.items():
                sm, solved = measure_solving(sol_fn, grid, n, seed,
                                             label=f"solve_{sol_name}", gen_label=f"gen_{gen_name}")
                rows.append(sm)

import pandas as pd
df = pd.DataFrame(rows)
df.head()

In [ ]:
# Sauvegarde CSV
out_csv = "maze_metrics.csv"
df.to_csv(out_csv, index=False)
print(f"✅ Résultats écrits dans {out_csv}")

In [ ]:
# Graphiques simples
plt.figure(figsize=(8,5))
for algo in df['algo'].unique():
    sub = df[(df['phase']=='generation') & (df['algo']==algo)]
    plt.plot(sub['n'], sub['wall_time_s'], marker='o', label=algo)
plt.xlabel('n')
plt.ylabel('Wall time (s)')
plt.title('Temps de génération')
plt.legend()
plt.show()